# Binary Search Tree FAQ Summary

[@mjd507][1] ｜ [WeChat Channel（简体中文版）][2]

![Logo](images/BST-1.png)

[1]: https://github.com/mjd507
[2]: https://mp.weixin.qq.com/s/WL-tJvWO5UNNm_WKCr7IVw

I wrote a summary of the [Binary Tree FAQ](Binary_Tree.ipynb) before, this time I compiled the *binary search tree*.

The *binary search tree* has one more feature on the basis of the *binary tree*:

- For a node, the value of its left subnode is less than that of the current node. Also, the value of its right subnode is grater than that of the current node.

The *binary search tree* is used because its search effeciency - which can be reduced to  $O(\log n)$.

Here I put four classic subject together and we can take the time to do, in order to deepen management solution.

- [Lookup nodes in a binary search tree](#Find-node)
- [Insert nodes in a binary search tree](#Insert-node)
- [Delete nodes in a binary search tree](#Delete-node)
- [The Kth largest element in the data stream](#The-Kth-largest-element-in-the-data-stream)

## Find node

> Given a binary search tree and a target value, find the node in the binary search tree which its value is equal to the target value.
>
> Returns the node and its subtree, or `None` if the node does not exist.
>
> For example: Given a target value of $2$ and the following binary search tree:
>
> ```text
>     4
>    / \
>   2   6
>  / \
> 1   3
> ```
>
> Return to this subtree:
> ```text
>   2
>  / \
> 1   3
> ```

In [1]:
def search(root: dict, value: int) -> dict:
    if root is None or root['value'] == value:
        return root
    elif root['value'] > value:
        return search(root['left'], value)
    else:
        return search(root['right'], value)

## Insert node

> Given a binary search tree and a value, insert the value into the binary search tree and keep the tree still a binary search tree.
>
> For example, given a node value of 5 and a binary search tree below:
> ```text
>     4
>    / \
>   2   7
>  / \
> 1   3
> ```
>
> Return to the following tree:
> ```text
>      4
>    /   \
>   2     7
>  / \   /
> 1   3 5
> ```

There provides two methods of recursion and iteration.

In [2]:
def insert_recursively(tree: dict, value: int) -> dict:
    if tree is None:
        return {'value': value}
    if tree['value'] > value:
        tree['left'] = tree.setdefault('left')
        tree['left'] = insert_recursively(tree['left'], value)
    else:
        tree['right'] = tree.setdefault('right')
        tree['right'] = insert_recursively(tree['right'], value)
    return tree

In [3]:
def insert_iteratively(tree: dict, value: int) -> dict:
    new_node = {'value': 5}
    if tree is None:
        return new_node
    dummy = tree
    previous = None
    while tree is not None:
        if tree['value'] > value:
            (previous, tree) = (tree, tree.setdefault('left'))
        else:
            (previous, tree) = (tree, tree.setdefault('right'))
    if previous['value'] > value:
        previous['left'] = new_node
    else:
        previous['right'] = new_node
    return dummy

## Delete node

> Delete a node in the binary search tree and keep the tree still a binary search tree.
>
> For example, given a node value of 3 and a binary search tree below:
> ```text
>     5
>    / \
>   3   6
>  / \   \
> 2   4   7
> ```
>
> <br>
>
> You should return the following binary tree:
> ```text
>     5                5
>    / \              / \
>   4   6      or    2   6
>  /     \            \   \
> 2       7            4   7
> ```

Deleting a node is more complicated. Assuming that the node to be deleted is $z$, there are three cases:
1. If $z$ has no child nodes, simply delete it.
2. If $z$ has only one child, then raise the child to the position of z.
3. If $z$ has two children, use the successor $y$ of $z$ to occupy the position of $z$.
    
    The successor $y$ of $z$ must be in the right subtree of $z$, because the right child of $z$ is not empty.
    
    $y$ also distinguish whether it is the direct right child of node $z$.

Here is [an article](http://www.algolist.net/Data_structures/Binary_search_tree/Removal) dedicated to the binary search tree deletion node.

The following is implemented according to its explanation.

In [4]:
class BSTNode:
    def __init__(self, value: int, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

    def minValue(self) -> int:
        if self.left is None:
            return self.value
        else:
            return self.left.minValue()

    def remove(self, value: int, parent) -> bool:
        if value < self.value:
            if self.left is not None:
                return self.left.remove(value, self)
            else:
                return False
        elif value > self.value:
            if self.right:
                return self.right.remove(value, self)
            else:
                return False
        else:
            # It has both left and right subtree.
            if self.left and self.right:
                self.value = self.right.minValue()
                self.right.remove(self.value, self)
            # It has only left subtree.
            elif parent.left == self:
                parent.left = (self.left if self.left else self.right)
            elif parent.right == self:
                parent.right = (self.left if self.left else self.right)
            return True

In [5]:
def delete(root: BSTNode, value: int) -> BSTNode:
    if root is None:
        return False
    if root.value == value:
        dummy_root = BSTNode(0, root, None)
        success = root.remove(value, dummy_root)
        root = dummy_root.left
        return success
    else:
        return root.remove(value, None)

## The Kth largest element in the data stream

> Design a class to find the kth largest element in the data stream.
>
> Ps: You only need to rank the element with the kth largest, and you don't need to filter the same element.
>
> The KthLargest class needs to have a constructor that accepts an integer `k` and an array of integers `arr` (initial elements). It also needs an `add(val)` method, adding one element at a time, returning the latest kth largest element.
>
> For example:
>
> ```python
> kthLargest = KthLargest(3, arr)
> # returns 4
> kthLargest.add(3)
> # returns 5
> kthLargest.add(5)
> # returns 5
> kthLargest.add(10)
> # returns 8
> kthLargest.add(9)
> # returns 8
> kthLargest.add(4)
> ```
>
> It can be assumed that the `arr` length is $\geq k-1$ and $k \geq 1$.

The core of this problem is solved by a minimum heap. First, construct a minimum heap when initializing. If the elements in the heap exceed `k`, directly pop out these smaller values, because these values will not play any role but only increase the complexity of the heap operation.

Python3 has a native implementation of the largest heap minimum heap. It is the core of this question. I recommend you write it yourself.

In [6]:
import heapq

class KthLargest:
    def __init__(self, k: int, arr: list):
        self.k = k
        self.heap = arr
        heapq.heapify(self.heap)

    def add(self, val: int) -> int:
        heapq.heappush(self.heap, val)
        return heapq.nlargest(self.k, self.heap)[-1]

In [7]:
largest = KthLargest(3, [4, 5, 8, 2])
largest.add(3)

4

In [8]:
largest.add(5)

5

In [9]:
largest.add(10)

5

In [10]:
largest.add(9)

8

In [11]:
largest.add(4)

8

## Afterword

I have maintained an algorithmic communication group. and the atmosphere is very good. If you want to join, scan the following QR code to add my personal WeChat account and I will pull you in.

![Wechat Account QR code](images/BST-2.jpg)